In [55]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

def get_nodes(df,colname):
    """
    Returns a set of nodes formed from column colname in dataframe df
    """
    n = set(df[colname].to_list())
    return n

def getEdges(df,n_colname,m_colname):
    """
    Returns a list of tuples with edges and edge attributes
    """    
    edges = []
    
    for i in range(df.shape[0]):
        node1 = df[n_colname].iloc[i]
        node2 = df[m_colname].iloc[i]
        
        gift_amt = df['Foreign Gift Amount'].iloc[i]
        gift_amt = gift_amt.replace(',','')
        gift_amt = gift_amt.replace('$','')
        gift_amt = float(gift_amt)
        
        gift_type = df['Gift Type'].iloc[i]
        
        contract_date = df['Foreign Gift Received Date'].iloc[i]
        
        data_dict = dict()
        data_dict["amt"] = gift_amt
        data_dict["type"] = gift_type
        data_dict["date"] = contract_date
        
        edges.append((node1,node2,data_dict))
    
    return edges

def createBipartiteGraph(df,n_colname,m_colname):
    """
    Creates a bipartite graph 
    """
    B = nx.MultiGraph()
    n = get_nodes(df,n_colname)
    m = get_nodes(df,m_colname)
    
    B.add_nodes_from(n, bipartite=0) # Add the node attribute "bipartite"
    B.add_nodes_from(m, bipartite=1)

    edges =  getEdges(df,n_colname,m_colname)
    B.add_edges_from(edges)
    
    return B

def plotBipartiteGraph(G):
    """
    Returns a set of nodes formed from column colname in dataframe df
    """
    # Separate by group
    l, r = nx.bipartite.sets(G)
    pos = {}

    # Update position for node from each group
    pos.update((node, (1, index)) for index, node in enumerate(l))
    pos.update((node, (2, index)) for index, node in enumerate(r))

    nx.draw(G, pos=pos)
    
    plt.figure(figsize=(40,40))
    plt.show()

In [56]:
df = pd.read_csv("Data/ForeignGifts/Foreign Gift-Table 1.csv")
df = df[['OPE ID','School Name','City','State','Foreign Gift Received Date','Foreign Gift Amount','Gift Type','Country of Giftor','Giftor Name']]
df.head()

OPE ID                            School Name          City State  \
0  110800                 University of Arkansas  Fayetteville    AR   
1  110800                 University of Arkansas  Fayetteville    AR   
2  110800                 University of Arkansas  Fayetteville    AR   
3  110800                 University of Arkansas  Fayetteville    AR   
4  131500  University of California, Los Angeles   Los Angeles    CA   

  Foreign Gift Received Date Foreign Gift Amount      Gift Type  \
0                 12/31/2015      $39,108,635.00       Contract   
1                 12/31/2015       $2,334,936.00       Contract   
2                 12/31/2015         $914,434.00       Contract   
3                 12/31/2015         $466,170.00       Contract   
4                 12/31/2015          $10,000.00  Monetary Gift   

  Country of Giftor                          Giftor Name  
0              IRAQ   Republic of Iraq, Higher Committee  
1            PANAMA  Instituto Para LA Formacion, Panama  
2      SAUDI ARABIA       Saudi Arabian Cultural Mission  
3             ITALY                         Giordana Srl  
4             JAPAN                     OKAWA FOUNDATION

In [57]:
print(df.shape)

(12966, 9)


In [58]:
n_colname = 'State'
m_colname = 'Country of Giftor'

# n = States
n = get_nodes(df,n_colname)
print("No. of nodes in n:")
print(len(n))

# m = Country
m = get_nodes(df,m_colname)
print("No. of nodes in m:")
print(len(m))

G = createBipartiteGraph(df,n_colname,m_colname)

#plotBipartiteGraph(G)

No. of nodes in n:
38
No. of nodes in m:
107


In [61]:
#Get all edges between two nodes
G['IL']['AFGHANISTAN']

AtlasView({0: {'amt': 1225689.0, 'type': 'Contract', 'date': '06/24/2014'}, 1: {'amt': 378900.0, 'type': 'Monetary Gift', 'date': '06/09/2014'}})

In [63]:
#Get attributes of specific edge between two nodes
G['IL']['AFGHANISTAN'][0]

{'amt': 1225689.0, 'type': 'Contract', 'date': '06/24/2014'}